In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F

import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
train_dataset = torchvision.datasets.FashionMNIST("./Data", download = True, train = True, transform = transforms.Compose([transforms.ToTensor()]))
test_dataset = torchvision.datasets.FashionMNIST("./Data", download = True, train = False, transform = transforms.Compose([transforms.ToTensor()]))

100.0%


Extracting ./Data\FashionMNIST\raw\train-images-idx3-ubyte.gz to ./Data\FashionMNIST\raw



100.0%


Extracting ./Data\FashionMNIST\raw\train-labels-idx1-ubyte.gz to ./Data\FashionMNIST\raw



100.0%


Extracting ./Data\FashionMNIST\raw\t10k-images-idx3-ubyte.gz to ./Data\FashionMNIST\raw



100.0%

Extracting ./Data\FashionMNIST\raw\t10k-labels-idx1-ubyte.gz to ./Data\FashionMNIST\raw

